# Canonical Identifier Analysis

This notebook explores the properties of the canonical identifier scheme used for public catalogue IDs.

## ID Generation Rules

Public catalogue identifiers follow specific rules to ensure they are:
- **Stable**: Once assigned, identifiers never change
- **Human-readable**: Easy to type and communicate
- **Unique**: No collisions in practical use

The scheme is defined as:
- **Length**: 8 characters
- **First character**: Letter only (`a-z`, excluding `o`, `i`, `l`)
- **Remaining characters**: Letters and digits (`a-z`, `2-9`, excluding `o`, `i`, `l`, `1`)

Confusing characters are excluded to avoid transcription errors (e.g., `0` vs `o`, `1` vs `l`).

In [1]:
from id_minter import generate_canonical_id

# Generate some example IDs
print("Example canonical IDs:")
for _ in range(5):
    print(f"  {generate_canonical_id()}")

Example canonical IDs:
  m2wzmxzf
  v88yu8vc
  ctutg49z
  jyevvcw7
  mznavpxm


## ID Space Size

How many unique IDs can we generate with this scheme?

- **First character**: 23 options (26 letters - 3 forbidden)
- **Remaining 7 characters**: 31 options each (26 letters + 9 digits - 4 forbidden)

Total: $23 \times 31^7 \approx 620$ billion unique identifiers

In [3]:
import string

def calculate_id_space(length: int = 8) -> int:
    """Calculate the total number of possible unique IDs."""
    forbidden = {'o', 'i', 'l', '1'}
    numbers = set(str(n) for n in range(1, 10))
    letters = set(string.ascii_lowercase)
    
    allowed_chars = (numbers | letters) - forbidden
    first_chars = letters - forbidden
    
    # First char options × (remaining char options)^(length-1)
    return len(first_chars) * (len(allowed_chars) ** (length - 1))

id_space = calculate_id_space()

print(f"Total ID space: {id_space:,}")
print(f"First character options: {26 - 3} (letters minus o, i, l)")
print(f"Other character options: {35 - 4} (digits 2-9 + letters minus forbidden)")
print(f"Formula: 23 × 31^7 = {23 * 31**7:,}")

Total ID space: 632,790,124,553
First character options: 23 (letters minus o, i, l)
Other character options: 31 (digits 2-9 + letters minus forbidden)
Formula: 23 × 31^7 = 632,790,124,553


## Collision Probability

When generating random IDs, what's the probability of a collision? This is the classic [birthday problem](https://en.wikipedia.org/wiki/Birthday_problem).

For $n$ items in an ID space of size $k$:
$$P(\text{collision}) \approx 1 - e^{-n^2/2k}$$

With ~630 billion possible IDs, collisions are extremely unlikely even at scale.

In [4]:
import math

def collision_probability(n_items: int, id_space: int) -> float:
    """
    Calculate collision probability using birthday problem approximation.
    P(collision) ≈ 1 - e^(-n²/2k) where n = items, k = ID space
    """
    exponent = -(n_items ** 2) / (2 * id_space)
    return 1 - math.exp(exponent)

def items_for_collision_probability(target_prob: float, id_space: int) -> int:
    """Calculate how many items needed to reach a target collision probability."""
    # From P = 1 - e^(-n²/2k), solve for n: n = sqrt(-2k * ln(1-P))
    return int(math.sqrt(-2 * id_space * math.log(1 - target_prob)))

id_space = calculate_id_space()

# Collision probabilities at different scales
print("Collision Probability (Birthday Problem):\n")
for n in [1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    prob = collision_probability(n, id_space)
    print(f"  {n:>12,} items: {prob:.2e} ({prob*100:.6f}%)")

print(f"\nItems needed for given collision probability:")
for target in [0.01, 0.1, 0.5]:
    n = items_for_collision_probability(target, id_space)
    print(f"  {target*100:>5.0f}% chance: {n:,} items")

print(f"\n50% collision threshold (birthday bound): ~{int(math.sqrt(id_space * math.pi / 2)):,}")

Collision Probability (Birthday Problem):

         1,000 items: 7.90e-07 (0.000079%)
        10,000 items: 7.90e-05 (0.007901%)
       100,000 items: 7.87e-03 (0.787038%)
     1,000,000 items: 5.46e-01 (54.622391%)
    10,000,000 items: 1.00e+00 (100.000000%)

Items needed for given collision probability:
      1% chance: 112,780 items
     10% chance: 365,160 items
     50% chance: 936,607 items

50% collision threshold (birthday bound): ~996,987
